In [54]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
#from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder

In [55]:
# Random seed for reproducibility
seed = 10
np.random.seed(seed)
# Import data
df = pd.read_csv('data/full_df.csv', index_col=0)
# Print first 10 samples
df.head()

,1,2,9,10,12,13,14,15,16,18,...,100630918,100631378,100642175,100652770,100652791,100653022,100689073,100689074,100820829,Beta
cell_line,,,,,,,,,,,,,,,,,,,,,
sw948,4.063884,3.594257,9.529689,6.740306,3.830637,3.751455,9.514933,3.385997,9.125257,6.499964,...,5.681741,3.452094,5.541555,5.345914,3.306113,4.295348,3.260120,3.234298,6.773936,10.0
mkn45,3.795708,3.587633,7.474039,4.230111,4.136858,8.231481,9.248871,3.343609,10.242179,4.155514,...,5.471486,3.456001,6.452955,4.510197,3.273790,4.282714,3.577665,3.363534,5.897890,7.0
j82,4.717745,3.816445,8.329890,4.542543,4.443213,3.834258,8.612918,3.683737,10.086658,5.139002,...,5.375150,3.474723,4.517609,4.268056,3.476972,3.994838,3.683059,3.375735,5.427906,3.0
sknfi,4.338183,3.749382,8.297267,5.911274,3.915953,3.699066,8.846982,3.763437,9.328351,7.383348,...,5.570602,3.541147,6.887191,4.065278,3.414457,3.906975,3.582676,3.187661,4.919210,3.0
te14,4.182824,3.627189,8.209574,3.672721,3.418317,5.969488,9.005162,3.404797,10.624380,4.212197,...,4.442833,3.514525,4.820618,4.510451,3.118535,4.811349,3.593005,6.203332,5.143104,6.0


In [56]:
df.isna().sum()

1            0
2            0
9            0
10           0
12           0
            ..
100653022    0
100689073    0
100689074    0
100820829    0
Beta         0
Length: 18524, dtype: int64

In [57]:
X = df.iloc[:,0:-1]
Y = df.iloc[:,-1]
print(X.shape)
print(Y.shape)

(444, 18523)
(444,)


In [58]:
X.describe()

,1,2,9,10,12,13,14,15,16,18,...,100628307,100630918,100631378,100642175,100652770,100652791,100653022,100689073,100689074,100820829
count,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,...,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000,444.000000
mean,4.759224,3.974918,8.054020,4.478085,4.939212,4.405639,8.832032,3.530190,10.123108,5.350088,...,3.615303,5.459427,3.566817,6.021262,4.457533,3.379636,4.381993,3.572660,3.835297,5.383554
std,0.904329,0.612940,1.122122,0.684921,2.042057,1.451931,0.454740,0.201927,0.681874,1.412162,...,0.184301,0.444793,0.178585,1.127011,0.382160,0.153203,0.432333,0.123645,1.042324,1.099485
min,3.053338,3.367974,3.867514,3.659009,3.418317,3.270635,7.160466,3.136587,7.868967,3.371530,...,3.077001,3.989987,3.227344,4.315654,3.720730,3.001211,3.620250,3.257888,2.927277,3.886451
25%,4.058643,3.672148,7.506647,4.137744,3.895449,3.603258,8.551606,3.407559,9.709321,4.171260,...,3.491003,5.173390,3.461360,5.151294,4.136312,3.269775,4.108727,3.494004,3.282901,4.565997
50%,4.400225,3.796596,8.128669,4.299478,4.130140,3.774769,8.847587,3.510482,10.114555,5.084553,...,3.609633,5.426343,3.534150,5.762099,4.453499,3.369673,4.301737,3.565830,3.403113,5.022973
75%,5.413454,3.973454,8.699563,4.545972,4.688962,4.362575,9.146902,3.623534,10.549979,6.301704,...,3.727752,5.678455,3.631855,6.662327,4.733341,3.473608,4.538599,3.655642,3.794921,6.024612
max,7.961095,6.995033,11.734768,9.941449,13.707150,11.435236,10.091407,6.050179,12.225875,10.513115,...,4.531748,7.258459,4.747194,10.495813,5.438839,3.820263,7.425968,3.982412,8.967523,9.434439


In [59]:
print(df.groupby(Y).size())

Beta
0.0     266
1.0      27
2.0      16
3.0      18
4.0      18
5.0      23
6.0      22
7.0      16
8.0      10
9.0       8
10.0      7
12.0     13
dtype: int64


In [60]:
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

In [61]:
Y = pd.get_dummies(Y)

In [62]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
X = X.values
Y = Y.values

In [72]:
# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model():
    # Create model here
    model = Sequential()
    model.add(Dense(15, input_dim = 48, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(15, activation = 'relu'))
    model.add(Dense(13, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [73]:
estimator = KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 10, verbose = 0)

C:\Users\Udi\AppData\Local\Temp\ipykernel_18956\1373799701.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 10, verbose = 0)


In [74]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = seed)

In [75]:
results = cross_val_score(estimator, X, Y, cv = kfold)
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\keras\wrappers\scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\keras\wrappers\scikit_learn.py", line 164, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
    raise e.with_traceba

Result: nan% (nan%)


C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\keras\wrappers\scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\keras\wrappers\scikit_learn.py", line 164, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\Udi\Documents\GitHub\RadiationProject\venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
    raise e.with_traceba